This notebook is for plotting interactive 3D figures with py3Dmol

In [1]:
import pickle 

import pandas as pd
from tqdm import tqdm

from rdkit import Chem

import py3Dmol
from ipywidgets import interact

import fresco

Load in the fragment pharmacophore distribution `frag_df`

In [2]:
interesting_pcores = ['Donor', 'Acceptor', 'Aromatic']

pharmacophores_to_atom_dict_for_coloring = {'Donor': 'O', 'Acceptor': 'N', 'Aromatic': 'Cl'}

mac1_dir = '/home/wjm41/ml_physics/frag-pcore-screen/data/Mac1'

frags = pickle.load(open(mac1_dir + '/frags_mac1.pickle', 'rb'))

frag_df = fresco.frag_funcs.return_pcore_dataframe(frags, interesting_pcores)
frag_df = frag_df[['pcore', 'coord_x', 'coord_y', 'coord_z']]
frag_df['pcore'] = [pharmacophores_to_atom_dict_for_coloring[x]
                    for x in frag_df['pcore']] 
frag_df['coord_x'] = frag_df['coord_x'].astype(str)
frag_df['coord_y'] = frag_df['coord_y'].astype(str)
frag_df['coord_z'] = frag_df['coord_z'].astype(str)


100%|██████████| 150/150 [00:06<00:00, 24.65it/s]


Plot the Mac1 protein

In [3]:
view = py3Dmol.view(width=800, height=800)

data_dir = '/home/wjm41/ml_physics/frag-pcore-screen/data'
file = open(mac1_dir+'/aligned/Mac1-DLS-EU0034_0A/Mac1-DLS-EU0034_0A_apo.pdb', 'r')
protein = file.read()
view.addModel(protein, 'pdb')

view.setStyle({'model': 0}, {'cartoon': {'color': 'yellow', 'opacity': 1.0}})
view.setBackgroundColor('white')
view.zoomTo()
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Plot the fragment ensemble in the binding site of Mpro (greyed out)

In [4]:
view = py3Dmol.view(width=800, height=800)

data_dir = '/home/wjm41/ml_physics/frag-pcore-screen/data'
file = open(mac1_dir+'/aligned/Mac1-DLS-EU0034_0A/Mac1-DLS-EU0034_0A_apo.pdb', 'r')
protein = file.read()
view.addModel(protein, 'pdb')

for mol in frags:
    mb = Chem.MolToMolBlock(Chem.RemoveHs(mol[0]), confId=-1)
    view.addModel(mb, 'sdf')
view.setStyle({'stick': {}})
view.setStyle({'model': 0}, {'cartoon': {'color': 'grey', 'opacity': 0.5}})
view.setBackgroundColor('white')
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Plot the pharmacophores equally spaced for use in a legend

In [5]:
pharmacophores_to_atom_dict_for_coloring = {'Donor': 'O', 'Acceptor': 'N', 'Aromatic': 'Cl'}

def plot_legend(r):
    view = py3Dmol.view(width=800, height=800)

    for n in range(3):

        legend = str(1)+'\n\n'
        legend = legend + \
            pharmacophores_to_atom_dict_for_coloring[list(pharmacophores_to_atom_dict_for_coloring)[n]]+' 0 ' + ' 0 '\
            + str(n)+'\n'
        view.addModel(legend, 'xyz')
        view.setStyle(
            {'model': -1}, {'sphere': {'radius': r, 'opacity': 0.95}})
    view.setBackgroundColor('0xffffff')
    view.zoomTo()
    view.show()
    return None


# interact(plot_legend, r=(0.1, 1.0, 0.1))  # (min, max, step)
plot_legend(r=0.1)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Plot Mac1 with all of the phramacophores

In [14]:
import numpy as np 

def plot_protein_with_all_pharmacophores(r, protein=False):
    view = py3Dmol.view()
    if protein:
        file = open(mac1_dir+'/aligned/Mac1-DLS-EU0034_0A/Mac1-DLS-EU0034_0A_apo.pdb', 'r')
        mac1 = file.read()
        view.addModel(mac1, 'pdb')
        view.setStyle({'model': -1}, {'cartoon': {'color': 'yellow', 'opacity': 0.5}})

    frag_pcores = str(len(frag_df))+'\n\n'
    vol_data = []
    for i, row in frag_df.iterrows():
        vol_data.append(row['pcore']+' '+row['coord_x'] +
                        ' '+row['coord_y']+' '+row['coord_z'])
        frag_pcores = frag_pcores + \
            row['pcore']+' '+row['coord_x']+' ' + \
            row['coord_y']+' '+row['coord_z']+'\n'
    view.addModel(frag_pcores, 'xyz')
    view.setStyle({'model': -1}, {'sphere': {'radius': r, 'opacity': 0.95}})
    for ind, mol in enumerate(frags):
        if ind in [8]:
            mb = Chem.MolToMolBlock(Chem.RemoveHs(mol[0]), confId=-1)
            view.addModel(mb, 'sdf')
            view.setStyle({'model': -1},{'stick': {'opacity': 1.0}})

    view.setBackgroundColor('0xffffff')
#     view.addSurface(py3Dmol.MS, {'opacity': 1.0})
    view.zoomTo()
    view.show()
    return None


# interact(plot_protein_with_all_pharmacophores, r=(0.1, 1.0, 0.1), protein=True)  # (min, max, step)
plot_protein_with_all_pharmacophores(r=0.2, protein=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Show pharmacophores with hit

In [5]:
import numpy as np 


def plot_protein_with_all_pharmacophores(r, protein=False):
    view = py3Dmol.view()
    if protein:
        file = open(mac1_dir+'/mac1_hit_aligned.pdb', 'r')
        mac1_hit = file.read()
        view.addModel(mac1_hit, 'pdb')
        view.setStyle({'model': -1}, {'cartoon': {'color': 'grey', 'opacity': 0.3}})
    file = open(mac1_dir+'/mac1_ligand.pdb', 'r')
    mac1_hit = file.read()
    view.addModel(mac1_hit, 'pdb')
    view.setStyle({'model': -1}, {'stick': {}})
    
    frag_pcores = str(len(frag_df))+'\n\n'
    vol_data = []
    for i, row in frag_df.iterrows():
        vol_data.append(row['pcore']+' '+row['coord_x'] +
                        ' '+row['coord_y']+' '+row['coord_z'])
        frag_pcores = frag_pcores + \
            row['pcore']+' '+row['coord_x']+' ' + \
            row['coord_y']+' '+row['coord_z']+'\n'
    view.addModel(frag_pcores, 'xyz')
    view.setStyle({'model': -1}, {'sphere': {'radius': r, 'opacity': 0.7}})
    # for ind, mol in enumerate(frags):
    #     if ind in [0, 1, 4]:
    #         mb = Chem.MolToMolBlock(Chem.RemoveHs(mol[0]), confId=-1)
    #         view.addModel(mb, 'sdf')
    #         view.setStyle({'model': -1},{'stick': {'opacity': 1.0}})

    view.setBackgroundColor('0xffffff')
#     view.addSurface(py3Dmol.MS, {'opacity': 1.0})
    view.zoomTo()
    view.show()
    return None


# interact(plot_protein_with_all_pharmacophores, r=(0.1, 1.0, 0.1), protein=True)  # (min, max, step)
plot_protein_with_all_pharmacophores(r=0.2, protein=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [55]:
frag_list = []
for ind, mol in enumerate(frags):
        if ind in [0, 1, 4]:
            smiles = Chem.MolToSmiles(Chem.RemoveHs(mol[0]))
            frag_list.append(smiles)
frag_list

['COC(=O)c1ccc(S(N)(=O)=O)cc1',
 'O=C(NCCc1ccncc1)NC1CCCCC1',
 'CCNc1ccc(C#N)cn1']

In [53]:
from os import listdir

filenames = [ f for f in listdir('../../data/Mpro/frags/') if f[-5:]=='0.pdb']
df = pd.read_csv('../../data/Mpro/frags/hits_summary.csv')
# df
df['smiles'] = [Chem.MolToSmiles(Chem.MolFromSmiles(smi)) for smi in df['Compound SMILES']]
df.query('smiles.isin(@frag_list)')

,Dataset,Library Name,Compound SMILES,Modified Compound SMILES,Compound ID,Site,Confidence Annotation,Resolution (A),Occupancy estimate,Z-Peak,PDB Status,PDB Entry,Fragalysis,Zenodo Status,Unnamed: 14,smiles
3,Mpro-x0161,DSIpoised_DMSO,COC(=O)C=1C=CC(=CC1)S(=O)(=O)N,NaN,Z18197050,A - active,4 - High confidence,1.93,0.86,11.98,Deposited,5R80,online,Deposited,NaN,COC(=O)c1ccc(S(N)(=O)=O)cc1
8,Mpro-x0305,DSIpoised_DMSO,CCNC=1C=CC(C#N)=CN1,NaN,Z219104216,A - active,4 - High Confidence,1.31,0.50,5.93,Deposited,5R82,online,Deposited,NaN,CCNc1ccc(C#N)cn1
24,Mpro-x0540,DSIpoised_DMSO,O=C(NCCC=1C=CN=CC1)NC2CCCCC2,NaN,Z111507846,A - active,4 - High Confidence,1.80,0.48,4.84,Deposited,5REH,online,Deposited,NaN,O=C(NCCc1ccncc1)NC1CCCCC1


Highlights two pharmacophores from all the others for illustration of distance measurement

In [13]:
def plot_one_pcore_and_select_another(r, n, protein=False):
    view = py3Dmol.view()
    if protein:
        file = open(mpro_dir+'/6YB7_model.pdb', 'r')
        mpro = file.read()
        view.addModel(mpro, 'pdb')
        view.setStyle(
            {'model': -1}, {'cartoon': {'color': 'yellow', 'opacity': 0.5}})

    for i, row in frag_df.iterrows():
        frag_pcores = str(1)+'\n\n' + \
            row['pcore']+' '+row['coord_x']+' ' + \
            row['coord_y']+' '+row['coord_z']+'\n'
        view.addModel(frag_pcores, 'xyz')
        if i==0:
            view.setStyle(
                {'model': -1}, {'sphere': {'radius': r, 'color':'red', 'opacity': 1.0}})
        elif i==n:
            view.setStyle(
                {'model': -1}, {'sphere': {'radius': r, 'color': 'blue', 'opacity': 1.0}})
        else:
            view.setStyle(
                {'model': -1}, {'sphere': {'radius': r, 'color': 'grey', 'opacity': 0.75}})
    view.setBackgroundColor('0xffffff')

    view.zoomTo({'model':n})
    view.show()
    return None


# interact(plot_one_pcore_and_select_another, r=(0.1, 1.0, 0.1), n=(
#     0, len(frag_df)-1, 1), protein=False)  # (min, max, step)
plot_one_pcore_and_select_another(r=0.2, n=0, protein=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol